# Maximum Likelihood Estimation

1. Generate a toy classification problem dataset for the cartesian coordinate system with two classes (K=2).
    - Pick two random cluster center coordinates.
    - For each cluster center, generate 40 data points (samples) by adding random noise values to cluster center values.
    - Store dataset X along with ground truth label values for each sample.
    - Shuffle the dataset (randomize the order of samples in your dataset).
    - Separate your dataset X into training and test sets
2. Implement multivariate classification using maximum likelihood using your training set.
3. Report performance of your algorithm using your test set.

In [1]:
import numpy as np
import pandas as pd
import random as rn
import math

**Pick two random cluster center coordinates**

In [2]:
class_lables =("class1" , "class2")
centers_dict = {}
center1 = (1 , 2)
center2 = (2 , 3)
centers_dict[center1] = class_lables[0]
centers_dict[center2] = class_lables[1]
centers_dict


{(1, 2): 'class1', (2, 3): 'class2'}

**generate 40 data points (samples) by adding random noise values**

In [3]:
N = 40
np.random.seed(0)
point1_X = np.round(np.random.uniform(center1[0]-1 , center1[0]+1,N),3)
point2_X = np.round(np.random.uniform(center2[0]-1 , center2[0]+1,N),3)

point1_Y = np.round(np.random.uniform(center1[1]-1 , center1[1]+1,N),3)
point2_Y = np.round(np.random.uniform(center2[1]-1 , center2[1]+1,N),3)


**Store dataset X along with ground truth label values for each sample**

In [4]:
dataset = []
for index in range(N):    
    dataset.append([point1_X[index] , point1_Y[index] , class_lables[0]])
    dataset.append([point2_X[index] , point2_Y[index] , class_lables[1]])

**Shuffle the dataset**

In [5]:
np.random.shuffle(dataset)

**Separate your dataset X into training and test sets**

In [6]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set = dataset[:train_size]
test_set = dataset[train_size+1:] 


### Multivariate classification using maximum likelihood

In [7]:
def cal_covariance_matrix(dataset , mean_vector): 
    col_len = dataset.shape[1]
    result = np.zeros((col_len, col_len)) 
    for item  in dataset:        
        variance = np.matrix(item - mean_vector)
        temp = np.dot(np.transpose(variance) , variance)
        result += temp
    
    return (result / (len(dataset) - 1))   

In [8]:
dataset_class1 =  np.array([[ x[0],x[1]] for x in train_set if x[2]== "class1"])
mean_class1 = dataset_class1.mean(axis= 0)
sigma_mat1 = cal_covariance_matrix(dataset_class1, mean_class1)
prior1 =   np.mean([i[1] for i in dataset_class1])

dataset_class2 =  np.array([[ x[0],x[1]] for x in train_set if x[2]== "class2"])   
mean_class2 = dataset_class2.mean(axis= 0)
sigma_mat2 = cal_covariance_matrix(dataset_class2, mean_class2)   
prior2 = np.mean([i[1] for i in dataset_class2])

print(sigma_mat2)

[[ 0.29958924 -0.0759681 ]
 [-0.0759681   0.32641181]]


In [9]:
def discriminate_func(x,y,d, mean_class, sigma_mat , prior):
    first_term = -d/2 * math.log( math.pi * 2)
    
    determinant_sigma_mat = np.linalg.det(sigma_mat)
    second_term = 0.5 * math.log(determinant_sigma_mat) 
    
    mean_item = [x-mean_class[0] , y-mean_class[1]]
    mean_item_transpose = np.transpose(mean_item)
    inverted_sigma_mat = np.linalg.inv(sigma_mat)
    

    third_term =  1/2 * np.dot(np.dot(mean_item, inverted_sigma_mat) , mean_item_transpose)
    result = first_term - second_term - third_term + math.log(prior)
    return result
    

In [10]:
d = 2
num_of_correct_classification = 0
for test in test_set:
    cls = test[2]
    discriminant_1 = np.round( discriminate_func(test[0],test[1],d, mean_class1, sigma_mat1 , prior1), 2)
    discriminant_2 = np.round( discriminate_func(test[0],test[1],d, mean_class2, sigma_mat2 , prior2) ,2)

    if discriminant_1 > discriminant_2 and cls == "class1":
        num_of_correct_classification += 1
    elif discriminant_1 < discriminant_2 and cls == "class2":
        num_of_correct_classification += 1
        
    print("disc1:{} , disc2:{} , {} , x:{} , y:{}".format(discriminant_1 , discriminant_2 ,cls,test[0],test[1] ))

print("{}result{}".format("-" *20 , "-" *20))
print('{} correct classification of total {}'.format( num_of_correct_classification , len(test_set)))


disc1:-2.14 , disc2:-6.98 , class1 , x:1.665 , y:1.04
disc1:-0.37 , disc2:-1.98 , class1 , x:0.923 , y:2.47
disc1:-5.08 , disc2:-2.38 , class2 , x:2.953 , y:2.299
disc1:-1.34 , disc2:0.17 , class2 , x:1.489 , y:2.863
disc1:-3.23 , disc2:-0.04 , class2 , x:2.313 , y:2.872
disc1:-0.53 , disc2:-0.81 , class2 , x:1.204 , y:2.597
disc1:-0.44 , disc2:-4.82 , class1 , x:1.292 , y:1.531
disc1:-3.24 , disc2:-0.09 , class2 , x:2.334 , y:2.848
disc1:-2.3 , disc2:0.18 , class2 , x:1.323 , y:3.182
disc1:-1.33 , disc2:0.22 , class2 , x:1.592 , y:2.814
disc1:-0.02 , disc2:-1.95 , class1 , x:1.28 , y:2.152
disc1:-1.6 , disc2:0.31 , class1 , x:1.561 , y:2.924
disc1:-2.39 , disc2:0.46 , class2 , x:1.874 , y:3.003
disc1:-1.95 , disc2:0.36 , class2 , x:1.877 , y:2.858
disc1:-5.37 , disc2:-0.26 , class2 , x:1.318 , y:3.793
--------------------result--------------------
13 correct classification of total 15
